In [24]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Average
from keras_contrib.layers import CRF
import keras
# define documents
import numpy as np
import pandas as pd
import os
from tqdm import tqdm 
# import matplotlib.pyplot as plt

In [25]:
## tokenize the text..
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tknzr.tokenize(sentence)
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)]
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)

In [26]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [27]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [7]:
# setup env
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/samarthgoal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/samarthgoal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
## load the embeddings (glove twitter and get the metrics from bi-LSTM) 

In [28]:
dir_path = "/home/samarthgoal/language_vision/input/reddit/"

In [29]:
df_rage = pd.read_csv(os.path.join(dir_path,'processed_rage.csv'))
df_happy =  pd.read_csv(os.path.join(dir_path,'processed_happy.csv'))
df_gore =  pd.read_csv(os.path.join(dir_path,'processed_gore.csv'))
df_creepy =  pd.read_csv(os.path.join(dir_path,'processed_creepy.csv'))

In [30]:
# create a random balances dataset of all of the categories
length = np.min([len(df_rage),len(df_happy),len(df_creepy),len(df_gore)])

In [31]:
df_final = pd.concat([df_rage[:length], df_happy[:length], df_gore[:length], df_creepy[:length]], ignore_index=True)

In [32]:
# create a random balances dataset of all of the categories
Y_new = df_final['subreddit']

In [33]:
# encode the output
Y_new = le.fit_transform(Y_new) 

In [34]:
token_list = (df_final['title'].apply(get_tokens))

In [46]:
## join the filtered tokens back again to the keras tokenizer which would give vocalb words etc
from keras.preprocessing.text import Tokenizer
sentences = [' '.join(tokens) for tokens in token_list]

In [47]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(token_list)
vocab_size = len(t.word_index) + 1
vocab_size

4990

In [37]:
len(token_list)

2460

In [38]:
max_len = 0
for idx,tokens in enumerate(token_list):
    if len(tokens) > max_len:
        max_len = len(tokens)
        index = idx
print(max_len)

34


In [39]:
vocab_size = vocab_size
# integer encode the documents
encoded_docs = t.texts_to_sequences(sentences)
# pad documents to a max length of 4 words
max_length = max_len
X = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
y = Y_new

In [40]:
## now splitting into test and training data
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train, Y_test =  train_test_split(X, y,test_size =0.20,random_state= 4 )

In [41]:
Y_train.shape

(1968,)

In [42]:
embedding_path = "/home/samarthgoal/embeddings/glove.twitter.27B.100d.txt" ## change

In [43]:
# create the word2vec dict from the dictionary
def get_word2vec(file_path):
    file = open(embedding_path, "r")
    if (file):
        word2vec = dict()
        split = file.read().splitlines()
        for line in split:
            key = line.split(' ',1)[0] # the first word is the key
            value = np.array([float(val) for val in line.split(' ')[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid fiel path")

In [44]:
w2v = get_word2vec(embedding_path)

FileNotFoundError: [Errno 2] No such file or directory: '/home/samarthgoal/embeddings/glove.twitter.27B.100d.txt'

In [45]:
# get the embedding matrix from the embedding layer
from numpy import zeros
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = w2v.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

NameError: name 'w2v' is not defined

In [48]:
X_train.shape

(1968, 34)

In [ ]:
## main model
input = Input(shape=(max_len,))
model =  Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length,trainable=False)(input)
model =  Bidirectional( LSTM(units=100,return_sequences=True,dropout=0.25),merge_mode="mul")(model)
model = TimeDistributed( Dense(100, activation="relu"))(model)
model = Flatten()(model)
model = Dense(100, activation = 'relu')(model)
out =  (Dense(4, activation = 'softmax'))(model)

In [446]:
## sandbox
input = Input(shape=(max_len,))
# model = Sequential()
model =  Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length,trainable=False)(input)
# model.add(e)
model =  Bidirectional( LSTM(units=100,return_sequences=True,dropout=0.25),merge_mode="mul")(model)
# model =  Bidirectional( LSTM(units=100,return_sequences=True,dropout=0.25),merge_mode="concat")(model)
model = TimeDistributed( Dense(100, activation="relu"))(model)
# model = Lambda(lambda x: tf.reduce_mean(x, axis=1))(model)
model = Flatten()(model)
model = Dense(100, activation = 'relu')(model)
out =  (Dense(4, activation = 'softmax'))(model)
# crf = CRF(n_tags)  # CRF layer
## @TODO add CRF and character encoding layer..

In [447]:
model = Model(input,out)

In [448]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [449]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_112 (InputLayer)       (None, 34)                0         
_________________________________________________________________
embedding_115 (Embedding)    (None, 34, 100)           499000    
_________________________________________________________________
bidirectional_131 (Bidirecti (None, 34, 100)           160800    
_________________________________________________________________
time_distributed_76 (TimeDis (None, 34, 100)           10100     
_________________________________________________________________
flatten_26 (Flatten)         (None, 3400)              0         
_________________________________________________________________
dense_180 (Dense)            (None, 100)               340100    
_________________________________________________________________
dense_181 (Dense)            (None, 4)                 404       
Total para

model = Sequential()
e =  Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)
model.add(e)
model.add( Bidirectional( LSTM(100,dropout=0.20),merge_mode='concat'))
model.add( Dense(4, activation = 'softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])



In [450]:
hist = model.fit(X_train, Y_train,validation_split=0.25, nb_epoch = 10, verbose = 2)

/home/samarthgoal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 1476 samples, validate on 492 samples
Epoch 1/10
 - 62s - loss: 1.2721 - acc: 0.3936 - val_loss: 1.0072 - val_acc: 0.5732
Epoch 2/10
 - 3s - loss: 0.9522 - acc: 0.6118 - val_loss: 0.8152 - val_acc: 0.6565
Epoch 3/10
 - 3s - loss: 0.7529 - acc: 0.6999 - val_loss: 0.8005 - val_acc: 0.6992
Epoch 4/10
 - 3s - loss: 0.6760 - acc: 0.7337 - val_loss: 0.7393 - val_acc: 0.7175
Epoch 5/10
 - 3s - loss: 0.6048 - acc: 0.7615 - val_loss: 0.7382 - val_acc: 0.7317
Epoch 6/10
 - 3s - loss: 0.5542 - acc: 0.7873 - val_loss: 0.7525 - val_acc: 0.7378
Epoch 7/10
 - 3s - loss: 0.5025 - acc: 0.8089 - val_loss: 0.7295 - val_acc: 0.7337
Epoch 8/10
 - 3s - loss: 0.4514 - acc: 0.8347 - val_loss: 0.8188 - val_acc: 0.7256
Epoch 9/10
 - 3s - loss: 0.4024 - acc: 0.8516 - val_loss: 0.7636 - val_acc: 0.7500
Epoch 10/10
 - 3s - loss: 0.3880 - acc: 0.8462 - val_loss: 0.7974 - val_acc: 0.7398


plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [451]:
# evaluate the model <100 LSTM two biderctional>
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 74.593496


In [453]:
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in Y_pred])
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.72      0.73       129
           1       0.75      0.64      0.69       106
           2       0.76      0.89      0.82       127
           3       0.73      0.72      0.72       130

   micro avg       0.75      0.75      0.75       492
   macro avg       0.75      0.74      0.74       492
weighted avg       0.75      0.75      0.74       492
 



In [437]:
# evaluate the model <simple LSTMlayer>
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 71.341463


In [149]:
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in Y_pred])
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.68      0.71       129
           1       0.61      0.73      0.66       106
           2       0.80      0.88      0.84       127
           3       0.76      0.62      0.68       130

   micro avg       0.73      0.73      0.73       492
   macro avg       0.73      0.73      0.72       492
weighted avg       0.73      0.73      0.72       492
 



In [ ]:
### fuse the language and image vectors...
